### Biblioteca Pandas | Leitura do arquivo vendas.csv

In [4]:
import pandas as pd

# Caminho para o arquivo
file_path = '/Users/batista/Downloads/vendas.csv'

### Inicializando acumuladores

Vamos usar para armazenar resultados intermediários ou parciais durante o processamento de grandes volume de dados.

Especialmente quando usamos aas técnicas de chunking para processar partes menores.

In [5]:
produtos_quantidade = {}
pais_regiao_vendas = {}
produtos_vendas_mensais = {}

### Leitura em chunks

Como a memoria e limitada vamos utilizar a técnica de chunking para processar este grande volume de dados de mil em mil.

In [6]:
chunksize = 1000
for chunk in pd.read_csv('vendas.csv', chunksize=chunksize, parse_dates=['Order Date']):
    
    # Produto mais vendido por quantidade e canal
    chunk_grouped_produto = chunk.groupby(['Item Type', 'Sales Channel'])['Units Sold'].sum()
    for (produto, canal), quantidade in chunk_grouped_produto.items():
        if (produto, canal) not in produtos_quantidade:
            produtos_quantidade[(produto, canal)] = 0
        produtos_quantidade[(produto, canal)] += quantidade

    # País e Região com maior volume de vendas em valor
    chunk['Total Sales'] = chunk['Units Sold'] * chunk['Unit Price']
    chunk_grouped_pais_regiao = chunk.groupby(['Country', 'Region'])['Total Sales'].sum()
    for (pais, regiao), vendas in chunk_grouped_pais_regiao.items():
        if (pais, regiao) not in pais_regiao_vendas:
            pais_regiao_vendas[(pais, regiao)] = 0
        pais_regiao_vendas[(pais, regiao)] += vendas

    # Média de vendas mensais por produto
    chunk['Month'] = chunk['Order Date'].dt.to_period('M')
    chunk_grouped_produto_mes = chunk.groupby(['Item Type', 'Month'])['Total Sales'].sum()
    for (produto, mes), vendas in chunk_grouped_produto_mes.items():
        if produto not in produtos_vendas_mensais:
            produtos_vendas_mensais[produto] = {}
        if mes not in produtos_vendas_mensais[produto]:
            produtos_vendas_mensais[produto][mes] = 0
        produtos_vendas_mensais[produto][mes] += vendas

### Produtos mais vendidos

Este código encontra o produto que teve a maior quantidade de unidades vendidas, considerando o canl de vendas.

Aqui a função **max** e **key=dprodutos_quantidade.get** permite identificar facilmente qual chave tem o maior quantidade vendida

In [7]:
produto_mais_vendido = max(produtos_quantidade, key=produtos_quantidade.get)
print(f"O produto mais vendido foi {produto_mais_vendido[0]} pelo canal {produto_mais_vendido[1]}")

O produto mais vendido foi Cereal pelo canal Offline


### Pais e região com maior volume de vendas

Nesta parte do código o objetivo foi encontrar a combinação de pais e região que teve o maior volume de vendas em termos de valor monetário.

Esta sendo usado a função **max** com **key=pais_regiao_vendas.get** para encontrar a chave "Pais e Região".

In [8]:
maior_venda_pais_regiao = max(pais_regiao_vendas, key=pais_regiao_vendas.get)
print(f"A maior venda foi em {maior_venda_pais_regiao[0]}, {maior_venda_pais_regiao[1]}")

A maior venda foi em Rwanda, Sub-Saharan Africa


### Media de vendas mensais por produto

Temos duas etapas aqui. \
**Primeiro:** Esta sendo realizado o cálculo da média de vendas por produto.\
**Segundo:** Este e um loop for que percorre cada intem no dicionario e apresenta o resultado com **print**.

In [20]:
media_vendas_mensais = {produto: sum(vendas.values()) / len(vendas) for produto, vendas in produtos_vendas_mensais.items()}
for produto, media in media_vendas_mensais.items():
    print(f"A média mensal de vendas do produto {produto} é {media:.2f}")

A média mensal de vendas do produto Baby Food é 4121979264.57
A média mensal de vendas do produto Beverages é 766422870.63
A média mensal de vendas do produto Cereal é 3321895316.21
A média mensal de vendas do produto Clothes é 1764843963.33
A média mensal de vendas do produto Cosmetics é 7061430780.67
A média mensal de vendas do produto Fruits é 150599554.70
A média mensal de vendas do produto Household é 10793288157.96
A média mensal de vendas do produto Meat é 6816059248.59
A média mensal de vendas do produto Office Supplies é 10523648469.69
A média mensal de vendas do produto Personal Care é 1319238686.16
A média mensal de vendas do produto Snacks é 2464503531.16
A média mensal de vendas do produto Vegetables é 2487487415.14


### Outra forma de apresentar a Media de vendas mensais por produto

In [19]:
# Reformatando a saída das médias mensais
print(f"{'Produto':<20} | {'Média Mensal de Vendas':>30}")
print("=" * 50)
for produto, media in media_vendas_mensais.items():
    print(f"{produto:<20} | {media:>30,.2f}")

Produto              |         Média Mensal de Vendas
Baby Food            |               4,121,979,264.57
Beverages            |                 766,422,870.63
Cereal               |               3,321,895,316.21
Clothes              |               1,764,843,963.33
Cosmetics            |               7,061,430,780.67
Fruits               |                 150,599,554.70
Household            |              10,793,288,157.96
Meat                 |               6,816,059,248.59
Office Supplies      |              10,523,648,469.69
Personal Care        |               1,319,238,686.16
Snacks               |               2,464,503,531.16
Vegetables           |               2,487,487,415.14
